In [1]:
# First we construct the quantum circuit

import numpy as np
import pennylane as qml
from pennylane import expval, var
from matplotlib import pyplot as plt


dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def circuit(params):
    # |psi_0>: state preparation
    qml.RY(np.pi / 4, wires=0)
    qml.RY(np.pi / 3, wires=1)
    qml.RY(np.pi / 7, wires=2)
    
    qml.RZ(params[0], wires=0)
    qml.RZ(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])


    qml.RY(params[2], wires=1)
    qml.RX(params[3], wires=2)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])

    qml.RY(params[2], wires=2)
    qml.RX(params[3], wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    
    return qml.expval(qml.PauliZ(0))

init_params = np.array([0.4, -0.1, 0.5, 0.2])

In [ ]:
# Now we apply the Natural gradient descent optimizer
steps = 300
gd_cost = []
opt = qml.GradientDescentOptimizer(0.01)

theta = init_params
for _ in range(steps):
    theta = opt.step(circuit, theta)
    gd_cost.append(circuit(theta))
    
    
qng_cost = []
opt = qml.QNGOptimizer(0.01)

theta = init_params
for _ in range(steps):
    theta = opt.step(circuit, theta)
    qng_cost.append(circuit(theta)) 
    


plt.style.use("seaborn")
plt.plot(gd_cost, "b", label="Vanilla gradient descent")
plt.plot(qng_cost, "g", label="Quantum natural gradient descent")

plt.ylabel("Cost function value")
plt.xlabel("Optimization steps")
plt.legend()
plt.show()